In [1]:
### Importar bibliotecas necessárias ###
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import time
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import pyautogui as pag

In [2]:
### Carregar a planilha que foi extraída do Excel com os números das notas
planilhaDf = pd.read_excel('trocaoleo.xlsx')

In [8]:
### Visualizar se a planilha carregou corretamente
### Necessário visualizar as colunas Maquina, Oleo e Quantidade
planilhaDf.head()

,Maquina,Quantidade,Oleo
0,159698,20,GC
1,159612,45,GC
2,159613,85,GC
3,159494,15,GC
4,159608,10,GC


In [4]:
### Processo de Login com SAP WebGUI ###

## Abre janela para digitar usuário e senha
usuario = pag.prompt('Digita seu usuário')
senha = pag.prompt('Digita sua senha')

## criar o navegador com o webdriver do Chrome
navegador = webdriver.Chrome()

## Link de acesso do WebGUI do SAP da ZF
navegador.get("https://swdlp1.zf-world.com/sap/bc/gui/sap/its/webgui")

## Preencher as informações de Login e senha
navegador.find_element(By.XPATH, '//*[@id="sap-user"]').send_keys(usuario)
navegador.find_element(By.XPATH, '//*[@id="sap-password"]').send_keys(usuario)

## Clicar no botão para realizar o LOGIN
navegador.find_element(By.XPATH, '//*[@id="LOGON_BUTTON"]').click()

In [5]:
time.sleep(10)
navegador.switch_to.frame("ITSFRAME1");
navegador.find_element(By.XPATH, '//*[@title="Entrar código de transação"]').send_keys("IW21")
navegador.find_element(By.XPATH, '//*[@title="Entrar código de transação"]').send_keys(Keys.ENTER)

In [6]:
pos = 0
tamanhoPlaniha = len(planilhaDf)
wait = WebDriverWait(navegador, 10)

for i in planilhaDf.Maquina:
    time.sleep(2)
    ## Aguardar a tela carregar e digitar a categoria de ordem M3 e pressionar ENTER
    fastrack = WebDriverWait(navegador, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="M0:46:::2:22"]')))
    fastrack.send_keys("M3")
    time.sleep(1)
    navegador.find_element(By.XPATH, '//*[@id="M0:46:::2:22"]').send_keys(Keys.ENTER)
    
    ## Aguarda carregar a tela e digita todas as informações necessárias
    textoNota = "Abastecimento de Oleo " + str(planilhaDf.Oleo[pos])
    fastrack2 = WebDriverWait(navegador, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@title="Texto breve"]')))
    fastrack2.send_keys(textoNota)
    navegador.find_element(By.XPATH, '//*[@title="Nº equipamento"]').send_keys(str(planilhaDf.Maquina[pos]))
    navegador.find_element(By.XPATH, '//*[@title="Nome do autor da nota"]').send_keys("VICTOR")
    navegador.find_element(By.XPATH, '//*[@title="Nome do autor da nota"]').send_keys(Keys.ENTER)
    time.sleep(2)
    navegador.find_element(By.XPATH, '//*[@title="Gravar (Ctrl+S)"]').click()
    pos = pos + 1
    if pos > tamanhoPlaniha:
        pag.alert("Finalizado abertura das notas!")
        break